In [51]:
import openai
import os
openai.api_key = os.getenv("OPENAI_API_KEY")
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
from generate_robot_instruction import encode_task_generation_prompt, generate_task_data

In [5]:
message = encode_task_generation_prompt()
message

[{'role': 'user',
  'content': "You are serving as a task-generation helper for a given robot environment.\xa0\n\nHere is the information about the environment. The environment is called {Two Block World}\xa0\nThere is a 7DOF Franka robot with a parallel gripper.\xa0\nThere are two blocks, {block A} and {block B}, with randomized sizes and density in the environment.\nThe blocks are initialized at a random position on a table.\nContents in the {} are the names of the objects in the environment.\n\nCome up with 10 different tasks for the robot to perform.\n\nGenerate the response following the template below:\n### Task {i}: {task description}\nwhere {i} is the task number and {task description} is the task description.\n\n\nAn example task description is: {pick up the heavier block and place it on top of the lighter block}\n\nThe rules for task description:\n1. Only include the objects in the environment in the task description.\n2. The task description doesn't need to include all the o

In [6]:
output = openai.ChatCompletion.create(
    model="gpt-4",
    messages=message,
    temperature=1.0,
    top_p=1,
    max_tokens=512,)
response = output['choices'][0]['message']['content']

In [19]:
output.choices

[<OpenAIObject at 0x7f8dd9558ef0> JSON: {
   "index": 0,
   "message": {
     "role": "assistant",
     "content": "### Task 1: Pick up block B and place it directly next to block A without altering the initial orientation of block A*.\n### Task 2: Manipulate block A and block B such that they are on the opposite sides of the table*.\n### Task 3: Maximize the height by stacking the two blocks on top of each other*.\n### Task 4: Pick up the lighter block and place it on the ground beneath the table without touching the heavier block.\n### Task 5: Rotate block A 90 degrees without moving it from its initial position*.\n### Task 6: Identify which block is denser, then lift that block and place it on top of the other block*.\n### Task 7: Reorient blocks A and B such that they are at 180 degrees from each other with the heaviest block on the table and the lighter block on top of it*.\n### Task 8: Move both block A and block B to the corners of the table.\n### Task 9: Arrange blocks A and B 

In [21]:
import re
import string

In [31]:
def post_process_task_response(response):
    if response is None:
        return []
    ### Parse the response into list of tasks ###
    raw_tasks = re.split("###", response)
    print(raw_tasks)
    tasks = []
    for idx, task in enumerate(raw_tasks):
        # # if the decoding stops due to length, the last example is likely truncated so we discard it
        # if idx == len(raw_tasks) - 1 and response["finish_reason"] == "length":
        #     continue
        ##### Parse the response into task #####
        splitted_data = re.split(f"{idx}:\s+", task)
        if len(splitted_data) != 2:
            continue
        else:
            task = splitted_data[1].strip()

        ##### FILTER OUT Negative Examples #####
        # filter out too short or too long tasks
        if len(task.split()) <= 3 or len(task.split()) > 150:
            continue
        # filter based on keywords that are not suitable for language models.
        # filter those starting with punctuation
        if task[0] in string.punctuation:
            continue
        # filter those starting with non-english character
        if not task[0].isascii():
            continue
        tasks.append(task)
    return tasks

In [94]:
generate_task_data(num_tasks_to_generate=20)

Loaded 10 machine-generated tasks


### [[{'role': 'user', 'content': "You are serving as a task-generation helper for a given robot environment.\xa0\n\nHere is the information about the environment. The environment is called {Two Block World}\xa0\nThere is a 7DOF Franka robot with a parallel gripper.\xa0\nThere are two blocks, {block A} and {block B}, with randomized sizes and density in the environment.\nThe blocks are initialized at a random position on a table.\nContents in the {} are the names of the objects in the environment.\n\nCome up with 10 different tasks for the robot to perform.\n\nGenerate the response following the template below:\n###Task {i}: {task description}\nwhere {i} is the task number and {task description} is the task description.\n\n\nAn example task description is: {pick up the heavier block and place it on top of the lighter block}\n\nThe rules for task description:\n1. Only include the objects in the environment in the task description.\n2. The task description doesn't need to include all the















prompts: 100%|██████████| 1/1 [00:27<00:00, 27.36s/it]






 50%|█████     | 10/20 [00:27<00:27,  2.74s/it]

['', 'Task 1: Pick up block A and place it on the opposite side of the table.\n', "Task 2: Stack block B on top of block A without altering block A's original position*.\n", 'Task 3: Using the end effector, tap block A three times without moving block B from its initial location*.\n', "Task 4: Slide block B around the table's surface for a total distance of one meter*.\n", 'Task 5: Pick up block A and rotate it 360 degrees in the gripper before placing it down at its original position. \n', "Task 6: Move block B to block A's initial position without toppling block A over*.\n", 'Task 7: Pick up block A and place it directly on top of block B, stacked perfectly upright*.\n', 'Task 8: Use the parallel gripper to hand over block A from one side of the workspace to the other side in a straight line.\n', 'Task 9: Squeeze block A with increasing force without lifting it, until the force reading on the end effector reaches a pre-set level*.\n', 'Task 10: Switch places of blocks A and B, ensuri